<a href="https://colab.research.google.com/github/Alesnim/chat_seq/blob/master/seq2seq_with-pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import os
import csv
import nltk
import itertools
import operator
import pickle
import numpy as np    
from keras.preprocessing import sequence
from scipy import sparse, io
from numpy.random import permutation
import re
import gensim

path_vec = './gdrive/My Drive/all.norm-sz100-w10-cb0-it1-min100.w2v'

embb = gensim.models.KeyedVectors.load_word2vec_format(path_vec, binary=True, unicode_errors='ignore')

In [0]:
from google.colab import drive
drive.mount('./gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./gdrive


In [0]:
import numpy as np
np.random.seed(1234)  # for reproducibility
import pandas as pd
import os
import csv
import nltk
import itertools
import operator
import pickle
import numpy as np    
from keras.preprocessing import sequence
from scipy import sparse, io
from numpy.random import permutation
import re

Using TensorFlow backend.


In [0]:
text = open('./gdrive/My Drive/full_raw.txt', 'r').readlines()
q = open('./gdrive/My Drive/context.txt', 'w')
a = open('./gdrive/My Drive/answers.txt', 'w')

In [0]:
for i, raw_word in enumerate(text):
  if i % 2 == 0:
    q.write(raw_word)
  else:
    a.write(raw_word)

In [0]:
q.close()
a.close()

In [0]:
questions_file = './gdrive/My Drive/context.txt'
answers_file = './gdrive/My Drive/answers.txt'
vocabulary_file = './gdrive/My Drive/vocabulary_from_raw.txt'
padded_questions_file = './gdrive/My Drive/Padded_context.txt'
padded_answers_file = './gdrive/My Drive/Padded_answers.txt'
unknown_token = 'something'

vocabulary_size = 7000
max_features = vocabulary_size
maxlen_input = 50
maxlen_output = 50  # cut texts after this number of words

print ("Reading the context data...")
q = open(questions_file, 'r')
questions = q.read()
print ("Reading the answer data...")
a = open(answers_file, 'r')
answers = a.read()
all = answers + questions
print ("Tokenazing the answers...")
paragraphs_a = [p for p in answers.split('\n')]
paragraphs_b = [p for p in all.split('\n')]
paragraphs_a = ['start '+p+' end' for p in paragraphs_a]
paragraphs_b = ['start '+p+' end' for p in paragraphs_b]
paragraphs_b = ' '.join(paragraphs_b)
tokenized_text = paragraphs_b.split()
paragraphs_q = [p for p in questions.split('\n') ]
tokenized_answers = [p.split() for p in paragraphs_a]
tokenized_questions = [p.split() for p in paragraphs_q]



Reading the context data...
Reading the answer data...
Tokenazing the answers...


In [0]:
### Counting the word frequencies:
word_freq = nltk.FreqDist(itertools.chain(tokenized_text))
print ("Found %d unique words tokens." % len(word_freq.items()))
##
### Getting the most common words and build index_to_word and word_to_index vectors:
vocab = word_freq.most_common(vocabulary_size-1)
##
### Saving vocabulary:
with open(vocabulary_file, 'wb') as v:
  pickle.dump(vocab, v)

Found 48516 unique words tokens.


In [0]:
vocab[6000]

('машинами', 7)

In [0]:
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print ("Using vocabulary of size %d." % vocabulary_size)
print ("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], 
                                                                                     vocab[-1][1]))

# Replacing all words not in our vocabulary with the unknown token:
for i, sent in enumerate(tokenized_answers):
    tokenized_answers[i] = [w if w in word_to_index else unknown_token for w in sent]
   


  
for i, sent in enumerate(tokenized_questions):
    tokenized_questions[i] = [w if w in word_to_index else unknown_token for w in sent]

# Creating the training data:
X = np.asarray([[word_to_index[w] for w in sent] for sent in tokenized_questions])
Y = np.asarray([[word_to_index[w] for w in sent] for sent in tokenized_answers])

Q = sequence.pad_sequences(X, maxlen=maxlen_input)
A = sequence.pad_sequences(Y, maxlen=maxlen_output, padding='post')

with open(padded_questions_file, 'wb') as q:
    pickle.dump(Q, q)
    
with open(padded_answers_file, 'wb') as a:
    pickle.dump(A, a)

Using vocabulary of size 7000.
The least frequent word in our vocabulary is 'сходить' and appeared 6 times.


In [0]:
from keras.layers import Input, Embedding, LSTM, Dense, RepeatVector, Bidirectional, Dropout, merge
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence

import keras.backend as K
import numpy as np


word_embedding_size = 100
sentence_embedding_size = 300
dictionary_size = 7000
maxlen_input = 50
maxlen_output = 50
num_subsets = 1
Epochs = 100
BatchSize = 128  #  Check the capacity of your GPU
Patience = 0
dropout = .25
n_test = 100

In [0]:
! wget https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextskipgram_300_5_2018.tgz

--2018-12-24 10:46:04--  https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextskipgram_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 176.195.79.50
Connecting to rusvectores.org (rusvectores.org)|176.195.79.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 707427878 (675M) [application/x-gzip]
Saving to: ‘araneum_none_fasttextskipgram_300_5_2018.tgz’

araneum_none_fastte 100%[===================>] 674.66M  5.55MB/s    in 2m 4s   

2018-12-24 10:48:09 (5.43 MB/s) - ‘araneum_none_fasttextskipgram_300_5_2018.tgz’ saved [707427878/707427878]



In [0]:
! head ./glove.6B.100d.txt

head: cannot open './glove.6B.100d.txt' for reading: No such file or directory


In [0]:
sdfdsfasdff = [1,1,1,1,1,1,1]
str(sdfdsfasdff)
type(sdfdsfasdff)

list

In [0]:
import gensim
path_vec = './gdrive/My Drive/all.norm-sz100-w10-cb0-it1-min100.w2v'

embb = gensim.models.KeyedVectors.load_word2vec_format(path_vec, binary=True, 
                                                       unicode_errors='ignore')
with open('./gdrive/My Drive/word2vec.100.txt', 'w') as f:
  for _,word in enumerate(embb.vocab):
    line = ''
    line = word
    vector = embb.get_vector(word)
    vector_str = ' '
    for num in vector:
      vector_str.join(' ' + str(num))
    f.write(line + ' ' + vector_str)

In [0]:
! wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.ru.300.vec.gz

--2018-12-24 12:00:26--  https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.ru.300.vec.gz
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.20.21
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.20.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1306357571 (1.2G) [binary/octet-stream]
Saving to: ‘cc.ru.300.vec.gz’

cc.ru.300.vec.gz    100%[===================>]   1.22G  36.2MB/s    in 32s     

2018-12-24 12:00:58 (39.3 MB/s) - ‘cc.ru.300.vec.gz’ saved [1306357571/1306357571]



In [0]:
! gunzip ./cc.ru.300.vec.gz

In [0]:
! head ./cc.ru.300.vec

2000000 300
, -0.0326 -0.1499 0.0232 0.0029 -0.0709 -0.0078 -0.0751 -0.0190 -0.0197 -0.0015 0.1447 0.4037 0.0650 -0.0183 0.0353 -0.0157 0.0130 -0.0376 0.0386 0.0096 -0.0644 0.2843 -0.0720 0.0322 -0.0135 -0.0217 0.1510 0.0431 -0.0017 0.0021 0.0285 -0.0526 0.0079 -0.0050 0.0025 -0.0294 -0.1391 -1.1404 -0.3228 -0.0796 0.0221 0.0347 0.0124 -0.1609 -0.1860 0.1010 -0.0072 0.0562 0.0104 0.0879 0.0207 0.0001 0.0189 0.0232 -0.0212 -0.1040 -0.0341 -0.0033 -0.2091 0.0981 -0.4552 -0.1114 0.0309 0.0974 -0.1251 0.0788 -0.1059 -0.0393 -0.0023 -0.0597 -0.1107 0.0122 -0.0025 0.0319 -0.1282 -0.0260 -0.0666 0.0950 0.0201 0.0387 0.0137 0.1422 -0.0038 -0.0242 0.0055 -0.1737 0.1817 0.0828 -0.0963 0.0215 0.0611 0.0505 -0.0185 -0.0197 0.0769 0.0172 0.0435 0.0718 -0.3437 0.0018 -0.0690 0.0055 -0.0850 -0.0049 0.0581 0.0275 -0.0969 0.0155 -0.0914 -0.0026 -0.0026 -0.0130 0.0385 -0.0297 0.0362 -0.0304 -0.0083 0.0178 -0.0502 0.0323 0.1164 -0.0139 0.0184 0.0551 0.0086 0.0343 -0.0594 -0.0466 -0.1004 -0.0480 0.0045 0.